In [1]:
from pathlib import Path
import pandas as pd
import plotly.express as px

from utils import activity as au
from utils import plots as pu
from utils.config import (
    M_TO_MI_MULTIPLIER,
    M_TO_FT_MULTIPLIER
)

In [2]:
path = Path("../data/parquet_run_activities")
last_activity_file = au.get_last_activity_file(path)
au.print_activity_summary(last_activity_file)

Activity summary for ../data/parquet_run_activities/2026-01-07-11-57-50.parquet:
  Elapsed time: 0:56:44
  Distance: 8.67 kilometers
  Elevation gain: 307 meters
  Average pace: 6.5 min/km
  Average heart rate: 175.4 bpm
  Average cadence: 77.7 spm
  Average power: 333.6 watts


In [3]:
df = pd.read_parquet(last_activity_file)
df.head()

,accumulated_power,activity_type,cadence,distance,enhanced_altitude,enhanced_speed,fractional_cadence,heart_rate,position_lat,position_long,...,sub_sport,complete_cadence,elapsed_seconds,altitude_change,altitude_gain,cum_altitude_gain,gradient,percent_grade,grade_degrees,origin_file_name
0,156,running,53,5.13,57.8,2.000,0.5,100,38.343441,-122.003455,...,trail,53.5,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2026-01-07-11-57-50.fit
1,459,running,55,8.43,57.6,2.019,0.0,100,38.343448,-122.003506,...,trail,55.0,1.0,-0.2,0.0,0.0,-0.060606,-6.060606,-3.468229,2026-01-07-11-57-50.fit
2,725,running,55,10.65,57.6,3.361,0.0,101,38.343442,-122.003530,...,trail,55.0,2.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2026-01-07-11-57-50.fit
3,1142,running,55,14.03,57.4,3.361,0.0,102,38.343433,-122.003559,...,trail,55.0,3.0,-0.2,0.0,0.0,-0.059172,-5.917160,-3.386334,2026-01-07-11-57-50.fit
4,1591,running,80,16.45,57.4,3.361,0.5,103,38.343425,-122.003586,...,trail,80.5,4.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2026-01-07-11-57-50.fit


In [4]:
# Metric to imperial
df['distance_mi'] = df['distance'] * M_TO_MI_MULTIPLIER
df['cum_elevation_gain_ft'] = df['cum_altitude_gain'] * M_TO_FT_MULTIPLIER
df['elevation_ft'] = df['enhanced_altitude'] * M_TO_FT_MULTIPLIER
df['elevation_change_ft'] = df['altitude_change'] * M_TO_FT_MULTIPLIER

In [5]:
title_distance = f"{df['distance_mi'].iloc[-1]:.2f}mi"
title_sport = f"{df['sub_sport'].iloc[0]} {df['sport'].iloc[0]}"
title_file = f"{df['origin_file_name'].iloc[0]}"
title = f"{title_distance} {title_sport} - {title_file}"
fig = pu.plot_run(df, color_col='elevation_ft', title=title)
fig.update_layout(
    width=800,
    height=600
)
fig.show()

In [6]:
# Cumulative elevation gain vs. distance plot
fig = px.line(
    df,
    x='distance_mi',
    y='cum_elevation_gain_ft',
    labels={
        'distance_mi': "Distance (Miles)",
        'cum_elevation_gain_ft': "Cumulative Elevation Gain (Feet)",
    },
    title="Cumulative Elevation Gain vs. Distance",
)
fig.update_traces(
    name="Cumulative Elevation Gain",
    showlegend=True,
    marker=dict(color='steelblue')
)

# Elevation line
fig.add_scatter(
    x=df['distance_mi'],
    y=df['elevation_ft'],
    mode='lines',
    name="Elevation",
    line=dict(color='black', dash='dash'),
)
fig.update_layout(
    showlegend=True,
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01),
    width=800,
    height=500,
)
fig.show()